<a href="https://colab.research.google.com/github/rfaylona/big-data-challenge/blob/main/amzon_camera_vine_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amzn_Camera_Vine_Reviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [18]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
camera_reviews = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

camera_reviews.show()
camera_reviews.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

1801974

In [24]:
# checking and removing null values and load in col dependencies

from pyspark.sql.functions import col,isnan, when, count, desc
camera_reviews.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in camera_reviews.columns]
   ).show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|               2|          2|            2|          2|   2|                2|              3|        124|         58|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [25]:
# check dataset size

drp_camera_rev = camera_reviews.dropna()
print((drp_camera_rev.count(), len(drp_camera_rev.columns)))

(1801849, 15)


In [26]:
# match and clean dataframe to schema

drp_camera_rev.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [27]:
# review table

review_id_table = drp_camera_rev.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
review_id_table.orderBy(desc("total_votes")).show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3CSHZYGZT7JS6|          5|         5132|       5287|   N|
|R3SLJAG34OOCG5|          5|         5109|       5181|   N|
|R1Y0PA7SS1UX4N|          5|         4965|       5010|   N|
| R5O3RQJXIOBDC|          5|         4675|       4756|   N|
|R1KCF39XVQCDBJ|          5|         4491|       4553|   N|
|R25AGEGV14RLYG|          5|         3582|       3615|   N|
|R2L4Q5OURK8L7S|          5|         2878|       2923|   N|
| R4EOUJ5QS2EJP|          5|         2680|       2798|   N|
|R33LNUMEWSVG00|          5|         2607|       2673|   N|
|R1V23D8ZZWZCTH|          5|         2580|       2611|   N|
|R16KPMS065H1TF|          5|         2565|       2609|   N|
|R1C7IFG69T6MVK|          5|         2535|       2589|   N|
|R3AMTDCW73QB03|          5|         2529|       2588|   N|
|R36NNUXNMQ8EKZ|          1|         246

In [28]:
# reduce size of dataframe for analysis by removing votes with < 100 votes

total_votes_rev = review_id_table.filter(review_id_table['total_votes'] < 100)
total_votes_rev.count()

1794644

In [29]:
# further reduce dataframe by filtering helpful votes with a score of 60%
helpful_votes_rev = total_votes_rev.filter(total_votes_rev['helpful_votes'] > 0.6)
helpful_votes_rev.count()

700577

**With reduced dataframe prepare for analysis**

In [31]:
helpful_votes_rev.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [34]:
# sperate reviews as non-vine member and as a vine member
member_rev = helpful_votes_rev.filter(helpful_votes_rev['vine']== 'Y')
member_rev.count()

5153

In [35]:
non_member_rev = helpful_votes_rev.filter(helpful_votes_rev['vine']== 'N')
non_member_rev.count()

695424

In [36]:
# base analysis on 5 star reviews 

member_star_rev = member_rev[member_rev['star_rating']== 5].count()
member_num = member_rev.count()
percent_star_rev = float(member_star_rev) / float(member_num)

print(f'Number of member reviews {member_num}')
print(f'Number of member reviews with 5 stars {member_star_rev}')
print(f'Percentage of member reviews that are five stars {percent_star_rev * 100}%')

Number of member reviews 5153
Number of member reviews with 5 stars 2161
Percentage of member reviews that are five stars 41.93673588201048%


In [37]:
non_member_star_rev = non_member_rev[non_member_rev['star_rating']== 5].count()
non_member_num = non_member_rev.count()
non_percent_star_rev = float(non_member_star_rev) / float(non_member_num)

print(f'Number of member reviews {non_member_num}')
print(f'Number of member reviews with 5 stars {non_member_star_rev}')
print(f'Percentage of member reviews that are five stars {non_percent_star_rev * 100}%')

Number of member reviews 695424
Number of member reviews with 5 stars 364754
Percentage of member reviews that are five stars 52.45059129394441%


**Analysis and Summary of Finding**

Based on the analysis on a smaller set of data, there is a 10% diffrence of member and non-member 5 star reviews.

To me a 10% difference on authenticity is still quite high. Based purely on non solicited vs solicited reviews.

The determination of factors on diffrence is difficult to tell, as there are emotional and logical factors to consider which is not present in the dataset.